In [69]:
import pandas as pd
import numpy as np
import re, os, sqlite3, math
from collections import Counter
from datetime import datetime

pd.set_option('max_colwidth', 160)
pd.set_option('mode.chained_assignment', None)

base = 'C:/Users/gdoku/YandexDisk/Компания/07 Учётные регистры и первичные документы/accounting.db'

In [6]:
'''
Обработка выписки по банковскому счёту
'''

# Выбираем файлы
path = r'C:/Users/gdoku/YandexDisk/Компания/06 Договоры (прочие)/Договор банковского счёта Банк ВТБ (ПАО)/Выписки'
files = os.listdir(path)

# Считываем и обрабатываем их
columns = ['date', 'number', 'value',
           'description', 'bik', 'account', 'reciver']
counter = 0
for file in files:
    data = pd.read_csv(os.path.join(path, file), sep=';', encoding='cp1251',
                       header=0, skiprows=[1, 2])
    data = data.iloc[:, [1, 2, 4, 6, 7, 8, 9]]
    if counter == 0:
        statment = data.copy()
        counter += 1
    else:
        statment = statment.append(data, ignore_index=True)
statment = statment.drop_duplicates().reset_index(drop=True)
statment.columns = columns
statment.number = statment.number.apply(lambda x: str(x))
statment.date = pd.to_datetime(statment.date, format='%d.%m.%Y')
statment.date = statment.date.apply(
    lambda x: pd.Timestamp.strftime(x, '%Y-%m-%d'))
statment.value = statment.value.apply(lambda x: float(x))
statment.bik = statment.bik.apply(lambda x: str(int(x)))
statment.sort_values(by='date')

# Загружаем данные в базу
conn = sqlite3.connect(base)
c = conn.cursor()
payments = pd.DataFrame(c.execute('SELECT * FROM payments_list').fetchall())
payments.columns = ['document_id'] + columns
payments.date = pd.to_datetime(payments.date).apply(
    lambda x: pd.Timestamp.strftime(x, '%Y-%m-%d'))
statment = statment.merge(payments.iloc[:, :4], on=[
                          'date', 'number', 'value'], how='outer')
statment = statment[statment.document_id.isnull()].iloc[:, :-1]
max_doc_id = c.execute(
    'SELECT max(document_id) FROM documents').fetchall()[0][0]
statment['id'] = statment.index + max_doc_id
new_docs = list(statment[['id', 'date', 'number']
                         ].itertuples(index=False, name=None))
new_payments = list(statment[['id'] + columns[2:]
                             ].itertuples(index=False, name=None))
c.executemany('INSERT INTO documents (document_id, document_type_id, document_date, document_number)\
                VALUES (?, 1, ?, ?)', new_docs)
c.executemany('INSERT INTO payments VALUES (?, ?, ?, ?, ?, ?)', new_payments)
print('Платёжных поручений добавлено: {}'.format(len(new_docs)))
conn.commit()
conn.close()

Платёжных поручений добавлено: 0


In [11]:
conn = sqlite3.connect(base)
c = conn.cursor()
data = c.execute('SELECT * FROM payments_list').fetchall()
conn.commit()
conn.close()

In [36]:
old_base = 'C:/Users/gdoku/YandexDisk/Компания/07 Учётные регистры и первичные документы/Журнал хозяйственных операций за 2020 год.xlsm'
mappping = 'C:/Users/gdoku/YandexDisk/Компания/07 Учётные регистры и первичные документы/mapping.xlsx'
inc_balance = pd.read_excel(old_base, sheet_name='Аналитические счета').iloc[:-1,:10]
inc_balance = inc_balance.iloc[:,[0,1,2,3,8,9]]
inc_balance.columns = ['№', 's_ac', 's_name', 'name', 'deb', 'cr']
inc_balance = inc_balance.loc[((inc_balance.deb!=0)|(inc_balance.cr!=0))&(inc_balance.s_ac!=99)]
#inc_balance.to_excel(mappping, sheet_name='mapping', index=False)

In [109]:
inc_balance = pd.read_excel(mappping)
conn = sqlite3.connect(base)
c = conn.cursor()
accounts = pd.DataFrame(c.execute('SELECT * FROM accounts').fetchall()).iloc[:,:-1]
conn.commit()
conn.close()
accounts.columns = ['account', 'ac_name', 'sc_type_id']
inc_balance = inc_balance.merge(accounts, on='account')
loans = inc_balance[inc_balance.sc_type_id==3]
docs = inc_balance[inc_balance.sc_type_id==4]
other = inc_balance[(inc_balance.sc_type_id!=3)&(inc_balance.sc_type_id!=4)]
other

,№,s_ac,ac_name_x,sc_name,deb,cr,account,ac_name_y,sc_type_id
0,51002,51,Расчётные счета,Расчётный счёт в Банке ВТБ (ПАО),643135.43,0.00,510,Расчётные счета,1.0
78,68000,68,Расчёты по налогам и сборам,"Налог, уплачиваемый в связи с УСН",0.00,8710.00,680,Расчёты по налогам и сборам,7.0
79,68300,68,Расчёты по налогам и сборам,Пошлина за обращение в суд,10586.00,0.00,680,Расчёты по налогам и сборам,7.0
97,80000,80,Уставный капитал,Уставный капитал,0.00,10000.00,800,Уставный капитал,2.0
98,84000,84,Нераспределённая прибыль,Нераспределённая прибыль,0.00,122050.88,840,Нераспределённая прибыль,NaN
99,96001,96,Резервы предстоящих расходов,Резерв под вознаграждение управляющему,0.00,6600.00,960,Резервы предстоящих расходов,9.0
100,97001,97,Расходы будущих периодов,Право пользования системой Контрур.Экстерн с 20.12.2018 до 19.12.2020,4618.97,0.00,970,Расхды будущих периодов,8.0
101,97002,97,Расходы будущих периодов,Электронная подпись с 26.03.2019 по 26.03.2020,939.89,0.00,970,Расхды будущих периодов,8.0
102,97003,97,Расходы будущих периодов,Регистрация домена с 23.06.2019 по 23.06.2020,429.85,0.00,970,Расхды будущих периодов,8.0


In [106]:
conn = sqlite3.connect(base)
subcounts = pd.read_sql('SELECT * FROM subcounts',conn)
subcounts.sc_type_id = subcounts.sc_type_id.apply(lambda x: float(x))
subcounts = subcounts[(subcounts.sc_type_id!=3)&(subcounts.sc_type_id!=4)][['sc_type_id', 'sc_name']]
other = other[['sc_type_id', 'sc_name']].dropna()
#other.merge(subcounts, align_axis=1)
subcounts = other.merge(subcounts, how='outer', on=['sc_type_id', 'sc_name'])
subcounts.to_sql('subcounts', conn, if_exists='append', index=False)

In [193]:
loans = loans[loans.s_ac == 58]
l = loans.copy()
pattern = '\d\d\.\d\d\.\d{4}'
l['document_date'] = l.sc_name.apply(lambda x: pd.to_datetime(re.findall(
    pattern, x)[0], format='%d.%m.%Y') if len(re.findall(pattern, x)) == 1 else None)
pattern_num = '№ .+ от'
l['document_number'] = l.sc_name.apply(lambda x: re.findall(
    pattern_num, x)[0][2:-3] if len(re.findall(pattern_num, x)) == 1 else None)
pattern_num = '№ .+;'
l['document_number'] = l.apply(lambda x: re.findall(
    pattern_num, x.sc_name) if pd.isnull(x.document_number) else x.document_number, axis=1)
l['document_number'] = l.document_number.apply(
    lambda x: (x[0] if len(x)>0 else None) if str(type(x))!='<class \'str\'>' else x)
l

,№,s_ac,ac_name_x,sc_name,deb,cr,account,ac_name_y,sc_type_id,document_date,document_number
1,58101,58,Финансовые вложения,"Договор займа от 04.07.2018; 19,5%",807582.07,0.0,581,Предоставленные займы,3.0,2018-07-04,None
2,58102,58,Финансовые вложения,"Договор займа от 04.07.2018; 38,0%",169438.26,0.0,581,Предоставленные займы,3.0,2018-07-04,None
3,58203,58,Финансовые вложения,"Договор № 34445-42873-41325-32 от 12.12.2018; ООО ""ТеоремаТрейд""",77207.04,0.0,581,Предоставленные займы,3.0,2018-12-12,34445-42873-41325-32
4,58205,58,Финансовые вложения,Договор № 35922-13659-41325-30 от 29.12.2018; ИП Тугбаева Л.А.,42767.29,0.0,581,Предоставленные займы,3.0,2018-12-29,35922-13659-41325-30
5,58206,58,Финансовые вложения,Договор № 34205-47800-41325-32 от 29.12.2018; ИП Шаульская Н.М.,54476.00,0.0,581,Предоставленные займы,3.0,2018-12-29,34205-47800-41325-32
6,58207,58,Финансовые вложения,"Договор № 181227-01 от 27.12.2018; ООО ""НТК Бизнес""",300000.00,0.0,581,Предоставленные займы,3.0,2018-12-27,181227-01
7,58208,58,Финансовые вложения,"Договор № 35976-42873-41325-31 от 22.01.2019; ООО ""ТеоремаТрейд""",84933.27,0.0,581,Предоставленные займы,3.0,2019-01-22,35976-42873-41325-31
8,58210,58,Финансовые вложения,Договор № 35367-49078-41325-35 от 17.01.2019; ИП Горяев С.В.,89319.81,0.0,581,Предоставленные займы,3.0,2019-01-17,35367-49078-41325-35
9,58212,58,Финансовые вложения,Договор № 34137-47733-41325-38; ИП Мальков П.М.,158703.53,0.0,581,Предоставленные займы,3.0,NaT,№ 34137-47733-41325-38;
10,58216,58,Финансовые вложения,Договор № 37180-50903-41325-34; ИП Писаренко С.В.,113015.14,0.0,581,Предоставленные займы,3.0,NaT,№ 37180-50903-41325-34;


In [2]:
directory = r'C:/Users/gdoku/YandexDisk/Компания/'\
    +'06 Договоры (прочие)/Договор оказания услуг ООО Мани Френдс/'\
    +'Выписки'
files = pd.Series(os.listdir(directory))

statment_files = list(files[files.str.contains('Выписка')])
schedule_files = list(files[files.str.contains('Оферта')])

counter = 0
for file in statment_files:
    text = open(os.path.join(directory, file), 'r', encoding='utf-8').read()
    text = text.replace('\n', ';').replace(
        ';Успешно;', '\n').replace(';Успешно', '').split('\n')
    if counter == 0:
        statment = pd.DataFrame(text).rename(columns={0:'raw'}).drop_duplicates()
        counter += 1
    else:
        statment = pd.concat([statment, pd.DataFrame(text).rename(columns={0:'raw'}).drop_duplicates()]).reset_index(
                drop=True).drop_duplicates()
        
statment.raw = statment.raw.str.split(';')
statment['date'] = pd.to_datetime(statment.raw.apply(lambda x: x[0]), format='%d.%m.%Y')
statment['time'] = pd.to_datetime(statment.raw.apply(lambda x: x[0]+' '+x[1]), format='%d.%m.%Y %H:%M:%S')
statment['type'] = statment.raw.apply(lambda x: ' '.join(x[2].split(' ')[0:4]))
statment['amount'] = statment.raw.apply(lambda x: float(x[3][:-2].replace(' ', '')))
statment['description'] = statment.raw.apply(lambda x: x[4] if len(x)==5 else x[2])
statment = statment.sort_values(by='time').drop(columns=['raw','time']).reset_index(drop=True)

types = ['Возврат займа', 'Возврат процентов по займу',
                         'Вывод средств', 'Пополнение средств', 'Перевод средств из резерва']
statment = statment[statment.type.isin(types)].reset_index(drop=True)
statment['debit'] = None
statment['credit'] = None
statment['loan'] = None
statment['borrower'] = None
statment.loc[statment.type.str.contains('Вывод средств'), 'debit'] = 51002
statment.loc[statment.type.str.contains('Вывод средств'), 'credit'] = 55001
statment.loc[statment.type.str.contains('Пополнение средств'), 'debit'] = 55001
statment.loc[statment.type.str.contains('Пополнение средств'), 'credit'] = 51002
statment.loc[statment.type.str.contains('Возврат займа'), 'debit'] = 55001
statment.loc[statment.type.str.contains('Возврат займа'), 'credit'] = None
statment.loc[statment.type.str.contains('Возврат процентов по займу'), 'debit'] = 55001
statment.loc[statment.type.str.contains('Возврат процентов по займу'), 'credit'] = None
statment.loc[statment.type.str.contains('Перевод средств из резерва'), 'debit'] = None
statment.loc[statment.type.str.contains('Перевод средств из резерва'), 'credit'] = 55001

pattern_loan = re.compile('№\d{3,4}')
pattern_borrower = re.compile('Заемщик .+\. С')

statment.loan = None
statment.loc[~statment.type.isin(['Пополнение средств', 'Вывод средств']), 'loan'] = statment.description.apply(
    lambda x: str(re.findall(pattern_loan, x))[3:-2])

statment.borrower = None
statment.loc[~statment.type.isin(['Пополнение средств', 'Вывод средств',
                                  'Перевод средств из резерва']), 'borrower'] = statment.description.apply(
    lambda x: str(re.findall('«.+»',str(re.findall(pattern_borrower, x))[10:-5]))[3:-3])

counter = 0
for file in schedule_files:
    text = open(os.path.join(directory, file), 'r', encoding='utf-8').read()
    text = text.replace('\n', ';').replace(
        ';Успешно;', '\n').replace(';Успешно', '').split('\n')
    if counter == 0:
        schedule = text
#         pd.DataFrame(text).rename(columns={0:'raw'}).drop_duplicates()
        counter += 1
    else:
        break
#         schedule = pd.concat([schedule, pd.DataFrame(text).rename(columns={0:'raw'}).drop_duplicates()]).reset_index(
#                 drop=True).drop_duplicates()

pd.DataFrame(np.array(schedule[0].split(';')[11:-4]).reshape(-1,4))